In [10]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [11]:
def get_suit(underlying_card_value):
    return math.ceil(underlying_card_value / 13)

def get_card_black_jack_value(underlying_card_value):
    card_true_value = underlying_card_value % 13
    if (card_true_value == 1):
        return 11
    elif (card_true_value >= 11 or card_true_value == 0):
        return 10
    else:
        return card_true_value

In [12]:
# Deck of cards class

class Deck:
    def __init__(self, shuffle = True):
        # suit 1 = clubs
        # suit 2 = spades
        # suit 3 = hearts
        # suit 4 = diamond
        self.cards = np.arange(1, 53)
        if (shuffle):
            np.random.shuffle(self.cards)
        self.removed = np.empty(0)
        
    def reshuffle_deck(self):
        self.cards = np.arange(1, 53)
        np.random.shuffle(self.cards)
        self.removed = np.empty(0)
        
    def pop(self):
        popped_card = self.cards[0]
        self.cards = np.delete(self.cards, 0)
        np.append(self.removed, popped_card)
        return get_card_black_jack_value(popped_card)
    
    def get_num_cards(self):
        return len(self.cards)

In [13]:
# Decks of cards class

class Decks:
    def __init__(self, number_of_decks = 1, shuffle = True):
        self.number_of_decks = number_of_decks
        self.decks = np.array([ Deck() for i in range(number_of_decks) ])
            
    def reshuffle_decks(self):
        self.decks = np.array([ Deck() for i in range(number_of_decks) ])
        
    def pop(self):
        if (len(self.decks) == 0):
            reshuffle_decks()
            pop()
        if (len(self.decks[0].cards) > 0):
            return self.decks[0].pop()
        else:
            self.decks = np.delete(self.decks, 0)
            pop()
    
    def get_num_cards(self):
        num = 0
        for deck in self.decks:
            num += deck.get_num_cards()

In [14]:
# betting strategy
BASIC_STRATEGY = 'BASIC_STRATEGY'
CARD_COUNTING = 'CARD_COUNTING'
WONGING = 'WONGING'

# staking strategy
KELLY_CRITERION = 'KELLY_CRITERION'
OSCARS_GRIND = 'OSCARS_GRIND'
MARTINGALE_SYSTEM = 'MARTINGALE_SYSTEM'
PAROLI_SYSTEM = 'PAROLI_SYSTEM'

# actions
HIT = 'HIT'
STAY = 'STAND'
SPLIT = 'SPLIT'
DOUBLE_DOWN = 'DOUBLE_DOWN'
BUST = 'BUST'
START = 'START'

# actions cont.
YES = 'YES'
NO = 'NO'
RAND = 'RAND'

In [ ]:
# runner
def run(decks, bankroll, staking_strat, betting_strat, number_of_players, insurance, rounds): 
    print('Not implemented yet!')

In [ ]:
# Player action functions 

# hand obj = {
#     hand: [10,7],
#     bet: $10
# }   

# TODO: Expand this to have hand action be determined by strategy, i.e make this generic
def execute_player_actions(decks, dealer_hand, player_hand_obj, player_hands, index):
    hand_action = START
    while hand_action != STAND:
        hand_action = get_action_for_basic_strategy(dealer_hand[0], player_hand_obj)
        if (hand_action == HIT):
            player_hand_obj = add_card_to_hand(decks, player_hand_obj)
        if (hand_action == SPLIT):
            # appends split hand to player_hands in mem
            player_hand_obj = split_hand(decks, player_hand_obj, player_hands, index)
        if (hand_action == DOUBLE_DOWN):
            player_hand_obj = double_down_on_hand(decks, player_hand_obj)
            hand_action = STAND
    player_hands[index] = player_hand_obj 
    
def add_card_to_hand(decks, player_hand_obj):
    next_card = decks.pop()
    updated_hand_values = np.append(player_hand_obj['values'], next_card)
    player_hand_obj['values'] = updated_hand_values
    return player_hand_obj

def split_hand(decks, player_hand_obj, player_hands, index):
    player_hand_obj['values'] = np.delete(player_hand_obj['values'], 0)
    split_player_hand_obj = {
        'values': player_hand_obj['values'],
        'bet': player_hand_obj['bet']
    }
    
    player_hand_obj = add_card_to_hand(decks, player_hand_obj)
    split_player_hand_obj = add_card_to_hand(decks, player_hand_obj)
    
    player_hands[index] = player_hand_obj
    player_hands.append(split_player_hand_obj)
    
    return player_hand_obj

def double_down_on_hand(decks, player_hand_obj):
    player_hand_obj['bet'] = player_hand_obj['bet'] * 2
    return add_card_to_hand(decks, player_hand_obj)

In [15]:
# Basic strategy involves making the statistically optimal decision for each possible hand based on the player's cards and the dealer's visible card. This strategy minimizes the house edge.
# Hard Hands (no Ace or Ace counted as 1):

# 8 or less: Always hit.
# 9: Double down if the dealer's card is 3-6, otherwise hit.
# 10: Double down if the dealer's card is 2-9, otherwise hit.
# 11: Always double down.
# 12: Stand if the dealer's card is 4-6, otherwise hit.
# 13-16: Stand if the dealer's card is 2-6, otherwise hit.
# 17 or more: Always stand.

# Soft Hands (Ace counted as 11):

# 13-15: Hit.
# 16-18: Double down if the dealer's card is 2-6, otherwise hit.
# 19: Stand unless the dealer has a 6, in which case double down.
# 20-21: Always stand.

# Pairs:

# 2s, 3s, 7s: Split if the dealer’s card is 2-7, otherwise hit.
# 4s: Split if the dealer’s card is 5 or 6, otherwise hit.
# 5s: Double down if the dealer’s card is 2-9, otherwise hit.
# 6s: Split if the dealer’s card is 2-6, otherwise hit.
# 8s: Always split.
# 9s: Split if the dealer’s card is 2-6 or 8-9, otherwise stand.
# 10s: Never split.
# Aces: Always split.

def execute_basic_strategy(decks, dealer_hand, player_hands):
    # in the case of splitting a player can have multuple hands
    for index, player_hand_obj in enumerate(player_hands.to_list()):        
        execute_player_actions(decks, dealer_hand, player_hand_obj, player_hands, index)
           
def get_action_for_basic_strategy(dealer_up_card, player_hand_obj):
    player_hand_value = np.sum(player_hand_obj.values)
    # Determine if the hand is a pair
    if len(player_hand_obj.values) == 2 and player_hand_obj.values[0] == player_hand_obj.values[1]:
        return pair_strategy(dealer_up_card, player_hand_value)
    
    # Determine if the hand is a soft hand (contains an Ace counted as 11)
    if 11 in player_hand_obj.values and player_hand_value <= 21:
        return soft_strategy(dealer_up_card, player_hand_value)
    
    # If not pair or soft, it's a hard hand
    if 11 in player_hand_obj.values and player_hand_value > 21:
        player_hand_obj.values = np.where(player_hand_obj.values == 11, 1, player_hand_obj.values)
        return hard_strategy(dealer_up_card, player_hand_value)
    return hard_strategy(dealer_up_card, player_hand_value)
            
def pair_strategy(dealer_up_card, player_hand_value):
    if player_hand_value in [2, 3, 7]:
        if dealer_up_card in [2, 3, 4, 5, 6, 7]:
            return SPLIT
        else:
            return HIT
    elif player_hand_value == 4:
        if dealer_up_card in [5, 6]:
            return SPLIT
        else:
            return HIT
    elif player_hand_value == 5:
        if dealer_up_card in [2, 3, 4, 5, 6, 7, 8, 9]:
            return DOUBLE_DOWN
        else:
            return HIT
    elif player_hand_value == 6:
        if dealer_up_card in [2, 3, 4, 5, 6]:
            return SPLIT
        else:
            return HIT
    elif player_hand_value == 8:
        return SPLIT
    elif player_hand_value == 9:
        if dealer_up_card in [2, 3, 4, 5, 6, 8, 9]:
            return SPLIT
        else:
            return STAND
    elif player_hand_value == 10:
        return STAND
    elif player_hand_value == 1:  # Treat Ace as 1 here for pairs
        return SPLIT

def soft_strategy(dealer_up_card, player_hand_value):
    if player_hand_value in [13, 14, 15]:
        return HIT
    elif player_hand_value in [16, 17, 18]:
        if dealer_up_card in [2, 3, 4, 5, 6]:
            return DOUBLE_DOWN
        else:
            return HIT
    elif player_hand_value == 19:
        if dealer_up_card == 6:
            return DOUBLE_DOWN
        else:
            return STAND
    else:
        return STAND

def hard_strategy(dealer_up_card, player_hand_value):
    if player_hand_value <= 8:
        return HIT
    elif player_hand_value == 9:
        if dealer_up_card in [3, 4, 5, 6]:
            return DOUBLE_DOWN
        else:
            return HIT
    elif player_hand_value == 10:
        if dealer_up_card in [2, 3, 4, 5, 6, 7, 8, 9]:
            return DOUBLE_DOWN
        else:
            return HIT
    elif player_hand_value == 11:
        return DOUBLE_DOWN
    elif player_hand_value == 12:
        if dealer_up_card in [4, 5, 6]:
            return STAND
        else:
            return HIT
    elif player_hand_value in [13, 14, 15, 16]:
        if dealer_up_card in [2, 3, 4, 5, 6]:
            return STAND
        else:
            return HIT
    elif player_hand_value >= 17:
        return STAND
    